In [1]:
import yaml
from datasets import load_dataset
from transformers import AutoTokenizer

In [2]:
# Paramètres
with open("config.yml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

INPUT_FICHIER_WIKI_TEXTE = config["camins"]["texte"]["fichier_wiki_texte"]
OUTPUT_DOSSIER_TOKENIZAIRE = config["camins"]["tokenizaire"]["dossier_tokenizaire"]

In [3]:
# Cargar ensemble de donadas
ensemble_donadas = load_dataset("csv", data_files=INPUT_FICHIER_WIKI_TEXTE)

In [4]:
def obtenir_corpus_entrainament(ensemble_donadas, talha_lot: int = 1000):
    """
    Obtenir corpus d'entrainament en lots.
    """
    conjunt_entrenament = ensemble_donadas["train"]
    for idx_debuta in range(0, len(ensemble_donadas), talha_lot):
        exemples = conjunt_entrenament[idx_debuta : idx_debuta + talha_lot]
        yield [x.lower() for x in exemples["Texte"]]


# Obtenir corpus d'entrainament
corpus_entrenament = obtenir_corpus_entrainament(ensemble_donadas)

In [5]:
# Cargar tokenizer de basa
base_tokenizaire = AutoTokenizer.from_pretrained(
    "gpt2",
    do_lower_case=True,
    additional_special_tokens=["z'", "z-"],
)
if base_tokenizaire.pad_token is None:
    base_tokenizaire.add_special_tokens({"pad_token": "[PAD]"})

In [6]:
# Entrenar un tokenizer novèl
tokenizaire = base_tokenizaire.train_new_from_iterator(corpus_entrenament, 30000)

In [7]:
# Testar tokenizer
texte = ensemble_donadas["train"][10]["Texte"]
tokens = tokenizaire.tokenize(texte)
print(texte)
print(tokens)

cuelha (queuille en francés) z-es una comuna occitana dau departament dau puèi de doma e la region d'auvèrnhe ròse aups. en 1789 la comuna s'apelava queulhe.
['cuelha', 'Ġ(', 'queuille', 'Ġen', 'ĠfrancÃ©s', ')', 'Ġ', 'z-', 'es', 'Ġuna', 'Ġcomuna', 'Ġoccitana', 'Ġdau', 'Ġdepartament', 'Ġdau', 'ĠpuÃ¨i', 'Ġde', 'Ġdoma', 'Ġe', 'Ġla', 'Ġregion', 'Ġd', "'", 'auvÃ¨rnhe', 'ĠrÃ²se', 'Ġaups', '.', 'Ġen', 'Ġ1789', 'Ġla', 'Ġcomuna', 'Ġs', "'", 'apelava', 'Ġqueul', 'he', '.']


In [8]:
# Salvar lo tokenizaire entrenat
tokenizaire.save_pretrained(OUTPUT_DOSSIER_TOKENIZAIRE)

('../models/tokenizaire/tokenizer_config.json',
 '../models/tokenizaire/special_tokens_map.json',
 '../models/tokenizaire/vocab.json',
 '../models/tokenizaire/merges.txt',
 '../models/tokenizaire/added_tokens.json',
 '../models/tokenizaire/tokenizer.json')